In [1]:
# Importing the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# Ignore harmless warnings
import warnings
warnings.filterwarnings("ignore")
import pandasql as psql
# pip install pandasql

In [2]:
# Load the Auto Quote Ins data
AutoIns = pd.read_csv(r"C:\Users\ARUN KUMAR\Desktop\capstone_Project_1\Auto_Quote_Data_V2.0.csv", header=0)
AutoIns.head()

,Quote_Num,Agent_Type,Q_Creation_DT,Q_Valid_DT,Policy_Bind_DT,Region,Agent_Num,Policy_Type,HH_Vehicles,HH_Drivers,...,Sal_Range1,Sal_Range2,Coverage,Veh_Usage,Annual_Miles_Range,Vehicl_Cost_Range1,Vehicl_Cost_Range2,Re_Quote,Quoted_Premium,Policy_Bind
0,AQ-C-139212,EA,2020/04/25,2020/06/23,2020/05/23,C,2156,Car,3,3,...,> $ 25 K <= $ 40 K,> 25 K <= 40 K,Balanced,Commute,> 55 K,> $ 10 K <= $ 20 K,> 10 K <= 20 K,No,693.86,Yes
1,AQ-F-136117,EA,2020/02/21,2020/04/20,NaN,F,2153,Van,2,2,...,> $ 40 K <= $ 60 K,> 40 K <= 60 K,Balanced,Pleasure,> 7.5 K & <= 15 K,<= $ 10 K,<= 10 K,No,635.96,No
2,AQ-F-126801,EA,2020/06/19,2020/08/17,2020/07/12,F,2056,Truck,2,1,...,> $ 40 K <= $ 60 K,> 40 K <= 60 K,Basic,Commute,> 35 K & <= 45 K,> $ 10 K <= $ 20 K,> 10 K <= 20 K,No,780.64,Yes
3,AQ-E-143467,EA,2020/05/02,2020/06/30,2020/05/24,E,2138,Car,1,2,...,> $ 90 K,> 90 K,Basic,Pleasure,<= 7.5 K,<= $ 10 K,<= 10 K,No,723.15,Yes
4,AQ-C-143827,EA,2020/02/12,2020/04/11,2020/02/25,C,2327,Truck,3,1,...,<= $ 25 K,<= 25 K,Basic,Pleasure,> 35 K & <= 45 K,<= $ 10 K,<= 10 K,No,738.14,Yes


In [3]:
# Displaying Duplicate values with in dataset
AutoIns_dup = AutoIns[AutoIns.duplicated(keep='last')]
AutoIns_dup


,Quote_Num,Agent_Type,Q_Creation_DT,Q_Valid_DT,Policy_Bind_DT,Region,Agent_Num,Policy_Type,HH_Vehicles,HH_Drivers,...,Sal_Range1,Sal_Range2,Coverage,Veh_Usage,Annual_Miles_Range,Vehicl_Cost_Range1,Vehicl_Cost_Range2,Re_Quote,Quoted_Premium,Policy_Bind


In [4]:
# Convert the target variable data type into interger
AutoIns['Policy_Bind'] = AutoIns['Policy_Bind'].str.replace('Yes', '1')
AutoIns['Policy_Bind'] = AutoIns['Policy_Bind'].str.replace('No', '0')
AutoIns['Policy_Bind'] = AutoIns['Policy_Bind'].astype(int)


In [5]:
# Count the target or dependent variable by '0' & '1' and
# their proportion (> 10 : 1, then the dataset is imbalance dataset)
Policy_Bind_count = AutoIns.Policy_Bind.value_counts()
print('Class 0:', Policy_Bind_count[0])
print('Class 1:', Policy_Bind_count[1])
print('Proportion:', round(Policy_Bind_count[0] / Policy_Bind_count[1], 2), ': 1')
print('Total Records:', len(AutoIns))

Class 0: 113757
Class 1: 32502
Proportion: 3.5 : 1
Total Records: 146259


In [6]:
# Ranges and new column as 'QP_Range' from 'Quoted_premium'
AutoIns['QP_Range'] = pd.cut(AutoIns['Quoted_Premium'], [0, 800, 1000, 1200, 9999],
 labels=['0-800', '801-1000', '1001-1200', '>1200'])

In [7]:
# Dropping the variables which are impacting the target variable
AutoIns = AutoIns.drop(['Quote_Num', 'Agent_Num', 'Q_Creation_DT', 'Q_Valid_DT', 'Policy_Bind_DT',
 'Sal_Range1', 'Vehicl_Cost_Range1', 'Quoted_Premium'], axis=1)

In [8]:
AutoIns.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146259 entries, 0 to 146258
Data columns (total 20 columns):
 #   Column              Non-Null Count   Dtype   
---  ------              --------------   -----   
 0   Agent_Type          146259 non-null  object  
 1   Region              146259 non-null  object  
 2   Policy_Type         146259 non-null  object  
 3   HH_Vehicles         146259 non-null  int64   
 4   HH_Drivers          146259 non-null  int64   
 5   Driver_Age          146259 non-null  int64   
 6   Driving_Exp         146259 non-null  int64   
 7   Prev_Accidents      146259 non-null  int64   
 8   Prev_Citations      146259 non-null  int64   
 9   Gender              146259 non-null  object  
 10  Marital_Status      146259 non-null  object  
 11  Education           146259 non-null  object  
 12  Sal_Range2          146259 non-null  object  
 13  Coverage            146259 non-null  object  
 14  Veh_Usage           146259 non-null  object  
 15  Annual_Miles_Rang

In [9]:
# Display the columns of data
AutoIns.columns

Index(['Agent_Type', 'Region', 'Policy_Type', 'HH_Vehicles', 'HH_Drivers',
       'Driver_Age', 'Driving_Exp', 'Prev_Accidents', 'Prev_Citations',
       'Gender', 'Marital_Status', 'Education', 'Sal_Range2', 'Coverage',
       'Veh_Usage', 'Annual_Miles_Range', 'Vehicl_Cost_Range2', 'Re_Quote',
       'Policy_Bind', 'QP_Range'],
      dtype='object')

In [10]:
AutoIns = pd.get_dummies(AutoIns, columns=['Agent_Type', 'Region', 'Policy_Type', 'Gender',
 'Marital_Status', 'Education', 'Sal_Range2',
 'Coverage', 'Veh_Usage', 'Annual_Miles_Range',
 'Vehicl_Cost_Range2', 'Re_Quote', 'QP_Range'])
AutoIns.head()

,HH_Vehicles,HH_Drivers,Driver_Age,Driving_Exp,Prev_Accidents,Prev_Citations,Policy_Bind,Agent_Type_EA,Agent_Type_IA,Region_A,...,Vehicl_Cost_Range2_> 10 K <= 20 K,Vehicl_Cost_Range2_> 20 K <= 30 K,Vehicl_Cost_Range2_> 30 K <= 40 K,Vehicl_Cost_Range2_> 40 K,Re_Quote_No,Re_Quote_Yes,QP_Range_0-800,QP_Range_801-1000,QP_Range_1001-1200,QP_Range_>1200
0,3,3,43,26,0,0,1,1,0,0,...,1,0,0,0,1,0,1,0,0,0
1,2,2,30,13,0,0,0,1,0,0,...,0,0,0,0,1,0,1,0,0,0
2,2,1,25,8,0,0,1,1,0,0,...,1,0,0,0,1,0,1,0,0,0
3,1,2,43,26,0,0,1,1,0,0,...,0,0,0,0,1,0,1,0,0,0
4,3,1,40,23,0,0,1,1,0,0,...,0,0,0,0,1,0,1,0,0,0


In [11]:
AutoIns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146259 entries, 0 to 146258
Data columns (total 60 columns):
 #   Column                                Non-Null Count   Dtype
---  ------                                --------------   -----
 0   HH_Vehicles                           146259 non-null  int64
 1   HH_Drivers                            146259 non-null  int64
 2   Driver_Age                            146259 non-null  int64
 3   Driving_Exp                           146259 non-null  int64
 4   Prev_Accidents                        146259 non-null  int64
 5   Prev_Citations                        146259 non-null  int64
 6   Policy_Bind                           146259 non-null  int32
 7   Agent_Type_EA                         146259 non-null  uint8
 8   Agent_Type_IA                         146259 non-null  uint8
 9   Region_A                              146259 non-null  uint8
 10  Region_B                              146259 non-null  uint8
 11  Region_C                  

In [12]:
# Identify the dependent and Target variables
IndepVar = []
for col in AutoIns.columns:
    if col != 'Policy_Bind':
        IndepVar.append(col)
TargetVar = 'Policy_Bind'
x = AutoIns[IndepVar]
y = AutoIns[TargetVar]


In [13]:
# Splitting the dataset into train and test
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.30, random_state =43)

In [14]:
# Splitting the dataset into train and test
from sklearn.model_selection import train_test_split
x1_train, x1_test, y1_train, y1_test = train_test_split(x, y, test_size = 0.30, stratify=y,  random_state =43)

In [15]:
# Feature Scaling - Each independent variable is in different range. The process of transforming all the
# features in the given data set to a fixed range is known as ‘Scaling’
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train['Driver_Age'] = sc.fit_transform(x_train['Driver_Age'].values.reshape(-1, 1))
x_train['Driving_Exp'] = sc.fit_transform(x_train['Driving_Exp'].values.reshape(-1, 1))
x_test['Driver_Age'] = sc.fit_transform(x_test['Driver_Age'].values.reshape(-1, 1))
x_test['Driving_Exp'] = sc.fit_transform(x_test['Driving_Exp'].values.reshape(-1, 1))
# Convert to dataframe
x_train = pd.DataFrame(x_train)
x_test = pd.DataFrame(x_test)


In [16]:
# Feature Scaling - Each independent variable is in different range. The process of transforming all the
# features in the given data set to a fixed range is known as ‘Scaling’
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x1_train['Driver_Age'] = sc.fit_transform(x1_train['Driver_Age'].values.reshape(-1, 1))
x1_train['Driving_Exp'] = sc.fit_transform(x1_train['Driving_Exp'].values.reshape(-1, 1))
x1_test['Driver_Age'] = sc.fit_transform(x1_test['Driver_Age'].values.reshape(-1, 1))
x1_test['Driving_Exp'] = sc.fit_transform(x1_test['Driving_Exp'].values.reshape(-1, 1))
# Convert to dataframe
x1_train = pd.DataFrame(x1_train)
x1_test = pd.DataFrame(x1_test)

# Decision Tree

# Decision Tree with Random Sampling


In [17]:
# Build the decision tree model with random sampling 
from sklearn.tree import DecisionTreeClassifier 
AutoInsDT = DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini', max_depth=None, 
                                   max_features=None, max_leaf_nodes=None, min_impurity_decrease=0.0, 
                                   min_impurity_split=None, min_samples_leaf=1, min_samples_split=2,
                                   min_weight_fraction_leaf=0.0, random_state=None, splitter='best') 
AutoInsDT = AutoInsDT.fit(x_train,y_train)

In [18]:
# Predict the model with test data set 

y_pred = AutoInsDT.predict(x_test)

In [19]:
# Display confusion matrix and classifiction report 
from sklearn.metrics import classification_report, confusion_matrix 

print(confusion_matrix(y_test, y_pred)) 
print(classification_report(y_test, y_pred))

[[25378  8731]
 [ 7309  2460]]
              precision    recall  f1-score   support

           0       0.78      0.74      0.76     34109
           1       0.22      0.25      0.23      9769

    accuracy                           0.63     43878
   macro avg       0.50      0.50      0.50     43878
weighted avg       0.65      0.63      0.64     43878



In [20]:
# Evaluate the model performance by metrics 
from sklearn import metrics 
from sklearn.metrics import roc_curve, roc_auc_score 

# Model Accuracy: how often is the classifier correct? 
print("Accuracy:", (round(metrics.accuracy_score(y_test, y_pred) * 100, 2)), "%") 

# Model Precision: what percentage of positive tuples are labeled as such? 
print("Precision:", (round(metrics.precision_score(y_test, y_pred) * 100, 2)), '%') 

# Model Recall: what percentage of positive tuples are labelled as such? 
print("Recall:", (round(metrics.recall_score(y_test, y_pred) * 100, 2)), "%") 

# Model f1-score: weighted average of Precision & Recall 
print("f1-score:", (round(metrics.f1_score(y_test, y_pred) * 100, 2)), '%') 

# Area under ROC curve 
print('roc_auc_score:', round(roc_auc_score(y_test, y_pred), 3))

Accuracy: 63.44 %
Precision: 21.98 %
Recall: 25.18 %
f1-score: 23.47 %
roc_auc_score: 0.498


# Decision Tree with Over Sampling

In [31]:
# Random oversampling can be implemented using the RandomOverSampler class

from imblearn.over_sampling import RandomOverSampler

over = RandomOverSampler(sampling_strategy='minority')
x_over, y_over = over.fit_resample(x, y)

print(x_over.shape)
print(y_over.shape)



(227514, 59)
(227514,)


In [33]:
x_over.shape

(227514, 59)

In [34]:
# To know the count of major and minor classes

y_over.value_counts()

0    113757
1    113757
Name: Policy_Bind, dtype: int64

In [35]:
# Splitting the dataset into train and test

from sklearn.model_selection import train_test_split

xo_train, xo_test, yo_train, yo_test = train_test_split(x_over, y_over, test_size = 0.3, random_state=42)

In [36]:
# Feature Scaling - Each independent variable is in different range. The process of transforming all the
# features in the given data set to a fixed range is known as ‘Scaling’

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

#x_train = sc.fit_transform(x_train)
#x_test = sc.fit_transform(x_test)

xo_train['Driver_Age'] = sc.fit_transform(xo_train['Driver_Age'].values.reshape(-1, 1))
xo_train['Driving_Exp'] = sc.fit_transform(xo_train['Driving_Exp'].values.reshape(-1, 1))
xo_test['Driver_Age'] = sc.fit_transform(xo_test['Driver_Age'].values.reshape(-1, 1))
xo_test['Driving_Exp'] = sc.fit_transform(xo_test['Driving_Exp'].values.reshape(-1, 1))

# convert to dataframe

xo_train = pd.DataFrame(xo_train)
xo_test = pd.DataFrame(xo_test)

In [37]:
# To build the decision tree model with Under Sampling

from sklearn.tree import DecisionTreeClassifier

AutoInsDT_O = DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                                      max_depth=None, max_features=None, max_leaf_nodes=None,
                                      min_impurity_decrease=0.0, min_impurity_split=None,
                                      min_samples_leaf=1, min_samples_split=2,min_weight_fraction_leaf=0.0,
                                      random_state=None, splitter='best')

AutoInsDT_O = AutoInsDT_O.fit(xo_train, yo_train)

In [38]:
# Predict with test data

yo_pred = AutoInsDT_O.predict(xo_test)

In [39]:
# Display confusion matrix and classifiction report 

from sklearn.metrics import classification_report,confusion_matrix

print(confusion_matrix(yo_test, yo_pred))
print(classification_report(yo_test, yo_pred))

[[24282  9795]
 [ 2107 32071]]
              precision    recall  f1-score   support

           0       0.92      0.71      0.80     34077
           1       0.77      0.94      0.84     34178

    accuracy                           0.83     68255
   macro avg       0.84      0.83      0.82     68255
weighted avg       0.84      0.83      0.82     68255



In [29]:
# Evaluate the model performance by metrics

from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score

# Model Accuracy: how often is the classifier correct?
print("Accuracy:", (round(metrics.accuracy_score(yo_test, yo_pred) * 100, 2)), "%")

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:", (round(metrics.precision_score(yo_test, yo_pred) * 100, 2)), '%')

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:", (round(metrics.recall_score(yo_test, yo_pred) * 100, 2)), "%")

# Model f1-score: weighted average of Precision & Recall
print("f1-score:", (round(metrics.f1_score(yo_test, yo_pred) * 100, 2)), '%')

# Area under ROC curve
print('roc_auc_score:', round(roc_auc_score(yo_test, yo_pred), 3))

Accuracy: 82.49 %
Precision: 76.37 %
Recall: 94.17 %
f1-score: 84.34 %
roc_auc_score: 0.825


# Decision Tree with Stratified Sampling

In [40]:
# Build the decision tree model with random sampling
from sklearn.tree import DecisionTreeClassifier
AutoInsDT_S = DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
 max_depth=None, max_features=None, max_leaf_nodes=None,
 min_impurity_decrease=0.0, min_impurity_split=None,
 min_samples_leaf=1, min_samples_split=2,min_weight_fraction_leaf=0.0,
 random_state=None, splitter='best')
AutoInsDT_S = AutoInsDT_S.fit(x1_train,y1_train)
AutoInsDT_S = AutoInsDT_S.fit(x1_train,y1_train)


In [41]:
# Predict the model with test data set
y1_pred = AutoInsDT_S.predict(x1_test)


In [42]:
# Display confusion matrix and classifiction report
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y1_test, y1_pred))
print(classification_report(y1_test, y1_pred))

[[25377  8750]
 [ 7298  2453]]
              precision    recall  f1-score   support

           0       0.78      0.74      0.76     34127
           1       0.22      0.25      0.23      9751

    accuracy                           0.63     43878
   macro avg       0.50      0.50      0.50     43878
weighted avg       0.65      0.63      0.64     43878



In [43]:
# Evaluate the model performance by metrics
from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score
# Model Accuracy: how often is the classifier correct?
print("Accuracy:", (round(metrics.accuracy_score(y1_test, y1_pred) * 100, 2)), "%")
# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:", (round(metrics.precision_score(y1_test, y1_pred) * 100, 2)), '%')
# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:", (round(metrics.recall_score(y1_test, y1_pred) * 100, 2)), "%")
# Model f1-score: weighted average of Precision & Recall
print("f1-score:", (round(metrics.f1_score(y1_test, y1_pred) * 100, 2)), '%')
# Area under ROC curve
print('roc_auc_score:', round(roc_auc_score(y1_test, y1_pred), 3))

Accuracy: 63.43 %
Precision: 21.9 %
Recall: 25.16 %
f1-score: 23.41 %
roc_auc_score: 0.498


# Decision Tree with Under Sampling

In [44]:
# Random undersampling to balance the class distribution 

from imblearn.under_sampling import RandomUnderSampler

under = RandomUnderSampler(sampling_strategy = 'majority')
x_under, y_under = under.fit_resample(x, y)

print(x_under.shape)
print(y_under.shape)

(65004, 59)
(65004,)


In [45]:
y_under.value_counts()

0    32502
1    32502
Name: Policy_Bind, dtype: int64

In [46]:
# Splitting the dataset into train and test

from sklearn.model_selection import train_test_split

xu_train, xu_test, yu_train, yu_test = train_test_split(x_under, y_under, test_size=0.3, random_state=42)

In [47]:
# Feature Scaling - Each independent variable is in different range. The process of transforming all the
# features in the given data set to a fixed range is known as ‘Scaling’

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

xu_train['Driver_Age'] = sc.fit_transform(xu_train['Driver_Age'].values.reshape(-1,1))
xu_train['Driving_Exp'] = sc.fit_transform(xu_train['Driving_Exp'].values.reshape(-1,1))
xu_train['Driver_Age'] = sc.fit_transform(xu_train['Driver_Age'].values.reshape(-1,1))
xu_train['Driving_Exp'] = sc.fit_transform(xu_train['Driving_Exp'].values.reshape(-1,1))

# convert to dataframe

xu_train =  pd.DataFrame(xu_train)
xu_test = pd.DataFrame(xu_test)

In [48]:
# To build the decision tree model with Under sampling

from sklearn.tree import DecisionTreeClassifier

AutoInsDT_C = DecisionTreeClassifier(criterion='gini', splitter='best', max_depth=None, min_samples_split=2, 
                                     min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, 
                                     random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, 
                                     min_impurity_split=None, class_weight=None, ccp_alpha=0.0)

AutoInsDT_C = AutoInsDT_C.fit(xu_train, yu_train)

In [49]:
# Predict with test data

yu_pred = AutoInsDT_C.predict(xu_test)

In [50]:
# Display confusion matrix and classification report 

from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(yu_test, yu_pred))
print(classification_report(yu_test, yu_pred))

[[4924 4829]
 [4880 4869]]
              precision    recall  f1-score   support

           0       0.50      0.50      0.50      9753
           1       0.50      0.50      0.50      9749

    accuracy                           0.50     19502
   macro avg       0.50      0.50      0.50     19502
weighted avg       0.50      0.50      0.50     19502



In [41]:
# Evaluate the model performance by metrics

from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score

# Model Accuracy: how often is the classifier correct?
print("Accuracy:", (round(metrics.accuracy_score(yu_test, yu_pred) * 100, 2)), "%")

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:", (round(metrics.precision_score(yu_test, yu_pred) * 100, 2)), '%')

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:", (round(metrics.recall_score(yu_test, yu_pred) * 100, 2)), "%")

# Model f1-score: weighted average of Precision & Recall
print("f1-score:", (round(metrics.f1_score(yu_test, yu_pred) * 100, 2)), '%')

# Area under ROC curve
print('roc_auc_score:', round(roc_auc_score(yu_test, yu_pred), 3))

Accuracy: 50.09 %
Precision: 50.08 %
Recall: 53.75 %
f1-score: 51.85 %
roc_auc_score: 0.501


# Decision Tree - Combining Over Sampling and Under Sampling

In [53]:
# Combining Random Oversampling and Undersampling
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
# define oversampling strategy
over = RandomOverSampler(sampling_strategy=0.40)
# fit and apply the transform
x2, y2 = over.fit_resample(x, y)
print(x2.shape)
print(y2.shape)
# define undersampling strategy
under = RandomUnderSampler(sampling_strategy=0.40)
# fit and apply the transform
x3, y3 = under.fit_resample(x2, y2)
print(x3.shape)
print(y3.shape)


(159259, 59)
(159259,)
(159257, 59)
(159257,)


In [54]:
y3.value_counts()


0    113755
1     45502
Name: Policy_Bind, dtype: int64

In [55]:
# Splitting the dataset into train and test

from sklearn.model_selection import train_test_split

xc_train, xc_test, yc_train, yc_test = train_test_split(x3, y3, test_size = 0.30, random_state = 42)

In [56]:
# Feature Scaling - Each independent variable is in different range. The process of transforming all the
# features in the given data set to a fixed range is known as ‘Scaling’

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

xc_train['Driver_Age'] = sc.fit_transform(xc_train['Driver_Age'].values.reshape(-1, 1))
xc_train['Driving_Exp'] = sc.fit_transform(xc_train['Driving_Exp'].values.reshape(-1, 1))
xc_test['Driver_Age'] = sc.fit_transform(xc_test['Driver_Age'].values.reshape(-1, 1))
xc_test['Driving_Exp'] = sc.fit_transform(xc_test['Driving_Exp'].values.reshape(-1, 1))

# convert to dataframe

xc_train = pd.DataFrame(xc_train)
xc_test = pd.DataFrame(xc_test)

In [57]:
# To build the decision tree model with Under sampling

from sklearn.tree import DecisionTreeClassifier

AutoInsDT_C = DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                                     max_depth=None, max_features=None, max_leaf_nodes=None,
                                     min_impurity_decrease=0.0, min_impurity_split=None,
                                     min_samples_leaf=1, min_samples_split=2,min_weight_fraction_leaf=0.0,
                                     random_state=None, splitter='best')

AutoInsDT_C = AutoInsDT_C.fit(xc_train,yc_train)

In [58]:
# Predict with test data

yc_pred = AutoInsDT_C.predict(xc_test)

In [59]:
# Display confusion matrix and classifiction report

from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(yc_test, yc_pred))
print(classification_report(yc_test, yc_pred))

[[24814  9150]
 [ 6235  7579]]
              precision    recall  f1-score   support

           0       0.80      0.73      0.76     33964
           1       0.45      0.55      0.50     13814

    accuracy                           0.68     47778
   macro avg       0.63      0.64      0.63     47778
weighted avg       0.70      0.68      0.69     47778



In [60]:
# Evaluate the model performance by metrics

from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score

# Model Accuracy: how often is the classifier correct?
print("Accuracy:", (round(metrics.accuracy_score(yc_test, yc_pred) * 100, 2)), "%")

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:", (round(metrics.precision_score(yc_test, yc_pred) * 100, 2)), '%')

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:", (round(metrics.recall_score(yc_test, yc_pred) * 100, 2)), "%")

# Model f1-score: weighted average of Precision & Recall
print("f1-score:", (round(metrics.f1_score(yc_test, yc_pred) * 100, 2)), '%')

# Area under ROC curve
print('roc_auc_score:', round(roc_auc_score(yc_test, yc_pred), 3))

Accuracy: 67.8 %
Precision: 45.3 %
Recall: 54.86 %
f1-score: 49.63 %
roc_auc_score: 0.64


# Random Forest

# Random Forest with Random Sampling

In [61]:
# Build Random Forest classification model and Train the model using the training sets
from sklearn.ensemble import RandomForestClassifier
AutoInsRF = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
 criterion='gini', max_depth=3, max_features='auto',
 max_leaf_nodes=None, max_samples=None,
 min_impurity_decrease=0.0, min_impurity_split=None,
 min_samples_leaf=1, min_samples_split=2,
 min_weight_fraction_leaf=0.0, n_estimators=500,
 n_jobs=None, oob_score=False, random_state=0, verbose=0,
 warm_start=False)
AutoInsRF = AutoInsRF.fit(x_train, y_train)


In [62]:
# Predict the model with test data set
y1_pred = AutoInsRF.predict(x_test)

In [63]:
# Display confusion matrix and classifiction report
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y1_pred))
print(classification_report(y_test, y1_pred))

[[34109     0]
 [ 9769     0]]
              precision    recall  f1-score   support

           0       0.78      1.00      0.87     34109
           1       0.00      0.00      0.00      9769

    accuracy                           0.78     43878
   macro avg       0.39      0.50      0.44     43878
weighted avg       0.60      0.78      0.68     43878



In [64]:
# Evaluate the model performance by mtrics

from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score

# Model Accuracy: how often is the classifier correct?
print("Accuracy:", (round(metrics.accuracy_score(y_test, y_pred) * 100, 2)), "%")

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:", (round(metrics.precision_score(y_test, y_pred, average='micro') * 100, 2)), '%')

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:", (round(metrics.recall_score(y_test, y_pred, average='micro') * 100, 2)), "%")

# Model f1-score: weighted average of Precision & Recall
print("f1-score:", (round(metrics.f1_score(y_test, y_pred, average='micro') * 100, 2)), '%')

# Area under ROC curve
print('roc_auc_score:', round(roc_auc_score(y_test, y_pred), 3))


Accuracy: 63.44 %
Precision: 63.44 %
Recall: 63.44 %
f1-score: 63.44 %
roc_auc_score: 0.498


# Random Forest with Over Sampling

In [65]:
# Build Random Forest classification model and Train the model using the training sets

from sklearn.ensemble import RandomForestClassifier

AutoInsRF_O = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
criterion='gini', max_depth=None, max_features='auto',
max_leaf_nodes=None, max_samples=None,
min_impurity_decrease=0.0, min_impurity_split=None,
min_samples_leaf=1, min_samples_split=2,
min_weight_fraction_leaf=0.0, n_estimators=500,
n_jobs=None, oob_score=False, random_state=0, verbose=0,
warm_start=False)
AutoInsRF_O = AutoInsRF_O.fit(xo_train, yo_train)

In [66]:
# Predict the model with test data set

yo_pred = AutoInsRF_O.predict(xo_test)

In [67]:
# Display confusion matrix and classifiction report

from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(yo_test, yo_pred))
print(classification_report(yo_test, yo_pred))

[[33716   361]
 [ 2960 31218]]
              precision    recall  f1-score   support

           0       0.92      0.99      0.95     34077
           1       0.99      0.91      0.95     34178

    accuracy                           0.95     68255
   macro avg       0.95      0.95      0.95     68255
weighted avg       0.95      0.95      0.95     68255



In [57]:
# Evaluate the model performance by metrics

from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score

# Model Accuracy: how often is the classifier correct?
print("Accuracy:", (round(metrics.accuracy_score(yo_test, yo_pred) * 100, 2)), "%")

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:", (round(metrics.precision_score(yo_test, yo_pred, average='micro') * 100, 2)), '%')

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:", (round(metrics.recall_score(yo_test, yo_pred, average='micro') * 100, 2)), "%")

# Model f1-score: weighted average of Precision & Recall
print("f1-score:", (round(metrics.f1_score(yo_test, yo_pred, average='micro') * 100, 2)), '%')

# Area under ROC curve
print('roc_auc_score:', round(roc_auc_score(yo_test, yo_pred), 3))

Accuracy: 95.4 %
Precision: 95.4 %
Recall: 95.4 %
f1-score: 95.4 %
roc_auc_score: 0.954


# Random Forest with Stratified Sampling

In [68]:
# Splitting the dataset into train and test
from sklearn.model_selection import train_test_split
x2_train, x2_test, y2_train, y2_test = train_test_split(x, y, test_size = 0.30, stratify=y,  random_state =43)

In [69]:
# Build the random forest model with random sampling
from sklearn.ensemble import RandomForestClassifier
AutoInsRF_S = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
criterion='gini', max_depth=None, max_features='auto',
max_leaf_nodes=None, max_samples=None,
min_impurity_decrease=0.0, min_impurity_split=None,
min_samples_leaf=1, min_samples_split=2,
min_weight_fraction_leaf=0.0, n_estimators=500,
n_jobs=None, oob_score=False, random_state=0, verbose=0,
warm_start=False)
AutoInsRF_S = AutoInsRF_S.fit(x2_train,y2_train)
AutoInsRF_S = AutoInsRF_S.fit(x2_train,y2_train)

In [73]:
# Predict the model with test data set
y2_pred = AutoInsRF_S.predict(x2_test)


In [74]:
# Display confusion matrix and classifiction report
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y2_test, y2_pred))
print(classification_report(y2_test, y2_pred))

[[34120     7]
 [ 9748     3]]
              precision    recall  f1-score   support

           0       0.78      1.00      0.87     34127
           1       0.30      0.00      0.00      9751

    accuracy                           0.78     43878
   macro avg       0.54      0.50      0.44     43878
weighted avg       0.67      0.78      0.68     43878



In [72]:
# Evaluate the model performance by metrics
from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score
# Model Accuracy: how often is the classifier correct?
print("Accuracy:", (round(metrics.accuracy_score(y2_test, y2_pred) * 100, 2)), "%")
# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:", (round(metrics.precision_score(y2_test, y2_pred) * 100, 2)), '%')
# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:", (round(metrics.recall_score(y2_test, y2_pred) * 100, 2)), "%")
# Model f1-score: weighted average of Precision & Recall
print("f1-score:", (round(metrics.f1_score(y2_test, y2_pred) * 100, 2)), '%')
# Area under ROC curve
print('roc_auc_score:', round(roc_auc_score(y2_test, y2_pred), 3))

Accuracy: 77.77 %
Precision: 30.0 %
Recall: 0.03 %
f1-score: 0.06 %
roc_auc_score: 0.5


#    Random Forest with Under Sampling

In [81]:
# Build Random Forest classification model and Train the model using the training sets

from sklearn.ensemble import RandomForestClassifier

AutoInsRF_U = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
criterion='gini', max_depth=None, max_features='auto',
max_leaf_nodes=None, max_samples=None,
min_impurity_decrease=0.0, min_impurity_split=None,
min_samples_leaf=1, min_samples_split=2,
min_weight_fraction_leaf=0.0, n_estimators=500,
n_jobs=None, oob_score=False, random_state=0, verbose=0,
warm_start=False)
AutoInsRF_U = AutoInsRF_U.fit(xu_train, yu_train)

In [82]:
# Predict the model with test data set

yu_pred = AutoInsRF_U.predict(xu_test)

In [83]:
# Display confusion matrix and classifiction report

from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(yu_test, yu_pred))
print(classification_report(yu_test, yu_pred))

[[3889 5864]
 [3823 5926]]
              precision    recall  f1-score   support

           0       0.50      0.40      0.45      9753
           1       0.50      0.61      0.55      9749

    accuracy                           0.50     19502
   macro avg       0.50      0.50      0.50     19502
weighted avg       0.50      0.50      0.50     19502



In [84]:
# Evaluate the model performance by metrics

from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score

# Model Accuracy: how often is the classifier correct?
print("Accuracy:", (round(metrics.accuracy_score(yu_test, yu_pred) * 100, 2)), "%")

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:", (round(metrics.precision_score(yu_test, yu_pred, average='micro') * 100, 2)), '%')

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:", (round(metrics.recall_score(yu_test, yu_pred, average='micro') * 100, 2)), "%")

# Model f1-score: weighted average of Precision & Recall
print("f1-score:", (round(metrics.f1_score(yu_test, yu_pred, average='micro') * 100, 2)), '%')

# Area under ROC curve
print('roc_auc_score:', round(roc_auc_score(yu_test, yu_pred), 3)) 

Accuracy: 50.33 %
Precision: 50.33 %
Recall: 50.33 %
f1-score: 50.33 %
roc_auc_score: 0.503


# Random Forest - Combining Over Sampling and Under Sampling

In [109]:
# Build Random Forest classification model and Train the model using the training sets

# Combining Random Oversampling and Undersampling
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
# define oversampling strategy
over = RandomOverSampler(sampling_strategy=0.45)
# fit and apply the transform
x2, y2 = over.fit_resample(x, y)
print(x2.shape)
print(y2.shape)
# define undersampling strategy
under = RandomUnderSampler(sampling_strategy=0.45)
# fit and apply the transform
x3, y3 = under.fit_resample(x2, y2)
print(x3.shape)
print(y3.shape)

# Combining Random Oversampling and Undersampling 
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
# define oversampling strategy
over = RandomOverSampler(sampling_strategy=0.50)
# fit and apply the transform
x2, y2 = over.fit_resample(x, y)
print(x2.shape)
print(y2.shape)
# define undersampling strategy
under = RandomUnderSampler(C)
# fit and apply the transform
x3, y3 = under.fit_resample(x2, y2)
print(x3.shape)
print(y3.shape)



(164947, 59)
(164947,)
(164945, 59)
(164945,)


In [118]:
y3.value_counts()

0    113755
1     51190
Name: Policy_Bind, dtype: int64

In [113]:
# Splitting the dataset into train and test

from sklearn.model_selection import train_test_split

xc_train, xc_test, yc_train, yc_test = train_test_split(x3, y3, test_size = 0.30, random_state = 42)

In [114]:
# Feature Scaling - Each independent variable is in different range. The process of transfo
# features in the given data set to a fixed range is known as ‘Scaling’
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
xc_train['Driver_Age'] = sc.fit_transform(xc_train['Driver_Age'].values.reshape(-1, 1))
xc_train['Driving_Exp'] = sc.fit_transform(xc_train['Driving_Exp'].values.reshape(-1, 1))
xc_test['Driver_Age'] = sc.fit_transform(xc_test['Driver_Age'].values.reshape(-1, 1))
xc_test['Driving_Exp'] = sc.fit_transform(xc_test['Driving_Exp'].values.reshape(-1, 1))

In [115]:
xc_train = pd.DataFrame(xc_train)
xc_test = pd.DataFrame(xc_test)

In [116]:
# Build Random Forest classification model and Train the model using the training sets

from sklearn.ensemble import RandomForestClassifier

AutoInsRF_C = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
criterion='gini', max_depth=None, max_features='auto',
max_leaf_nodes=None, max_samples=None,
min_impurity_decrease=0.0, min_impurity_split=None,
min_samples_leaf=1, min_samples_split=2,
min_weight_fraction_leaf=0.0, n_estimators=500,
n_jobs=None, oob_score=False, random_state=0, verbose=0,
warm_start=False)
AutoInsRF_C = AutoInsRF_C.fit(xc_train,yc_train)


# Predict the model with test data set

yc_pred = AutoInsRF_C.predict(xc_test)

# Display confusion matrix and classifiction report

from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(yc_test, yc_pred))
print(classification_report(yc_test, yc_pred))

# Evaluate the model performance by metrics

from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score

# Model Accuracy: how often is the classifier correct?
print("Accuracy:", (round(metrics.accuracy_score(yc_test, yc_pred) * 100, 2)), "%")

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:", (round(metrics.precision_score(yc_test, yc_pred, average='micro') * 100, 2)), '%')

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:", (round(metrics.recall_score(yc_test, yc_pred, average='micro') * 100, 2)), "%")

# Model f1-score: weighted average of Precision & Recall
print("f1-score:", (round(metrics.f1_score(yc_test, yc_pred, average='micro') * 100, 2)), '%')

# Area under ROC curve
print('roc_auc_score:', round(roc_auc_score(yc_test, yc_pred), 3))

[[33951    44]
 [ 7840  7649]]
              precision    recall  f1-score   support

           0       0.81      1.00      0.90     33995
           1       0.99      0.49      0.66     15489

    accuracy                           0.84     49484
   macro avg       0.90      0.75      0.78     49484
weighted avg       0.87      0.84      0.82     49484

Accuracy: 84.07 %
Precision: 84.07 %
Recall: 84.07 %
f1-score: 84.07 %
roc_auc_score: 0.746


# Logistic Regression

# Logistic Regression with  Random Sampling

In [71]:
# To build the 'Logistic Regression' model with random sampling

from sklearn.linear_model import LogisticRegression

AutoInsLR = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
intercept_scaling=1, max_iter=100, multi_class='auto',
n_jobs=None, penalty='l2', random_state=None,
solver='lbfgs', tol=0.0001, verbose=0, warm_start=False)
AutoInsLR = AutoInsLR.fit(x_train,y_train)

In [72]:
# Predict the model with test data set

y_pred = AutoInsLR.predict(x_test)

In [73]:
# Display confusion matrix and classifiction report

from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[34109     0]
 [ 9769     0]]
              precision    recall  f1-score   support

           0       0.78      1.00      0.87     34109
           1       0.00      0.00      0.00      9769

    accuracy                           0.78     43878
   macro avg       0.39      0.50      0.44     43878
weighted avg       0.60      0.78      0.68     43878



In [74]:
# Evaluate the model performance by metrics

from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score

# Model Accuracy: how often is the classifier correct?
print("Accuracy:", (round(metrics.accuracy_score(y_test, y_pred) * 100, 2)), "%")

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:", (round(metrics.precision_score(y_test, y_pred, average='micro') * 100, 2)), '%')

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:", (round(metrics.recall_score(y_test, y_pred, average='micro') * 100, 2)), "%")

# Model f1-score: weighted average of Precision & Recall
print("f1-score:", (round(metrics.f1_score(y_test, y_pred, average='micro') * 100, 2)), '%')

# Area under ROC curve
print('roc_auc_score:', round(roc_auc_score(y_test, y_pred), 3))

Accuracy: 77.74 %
Precision: 77.74 %
Recall: 77.74 %
f1-score: 77.74 %
roc_auc_score: 0.5


# Logistic Regression with Over Sampling

In [76]:
# To bulid the 'Logistic Regression' model with oversampling

from sklearn.linear_model import LogisticRegression

AutoInsLR_O = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
intercept_scaling=1, max_iter=100, multi_class='auto',
n_jobs=None, penalty='l2', random_state=None,
solver='lbfgs', tol=0.0001, verbose=0, warm_start=False)

AutoInsLR_O = AutoInsLR_O.fit(xo_train,yo_train)

In [77]:
# Predict the model with test data set

yo_pred = AutoInsLR_O.predict(xo_test)


In [78]:
# Display the confusion matrix and classification report

from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(yo_test, yo_pred))
print(classification_report(yo_test, yo_pred))

[[16548 17529]
 [16175 18003]]
              precision    recall  f1-score   support

           0       0.51      0.49      0.50     34077
           1       0.51      0.53      0.52     34178

    accuracy                           0.51     68255
   macro avg       0.51      0.51      0.51     68255
weighted avg       0.51      0.51      0.51     68255



In [79]:
# Evaluate the model performance by metrics

from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score

# Model Accuracy: how often is the classifier correct?
print("Accuracy:", (round(metrics.accuracy_score(yo_test, yo_pred) * 100, 2)), "%")

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:", (round(metrics.precision_score(yo_test, yo_pred, average='micro') * 100, 2)), '%')

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:", (round(metrics.recall_score(yo_test, yo_pred, average='micro') * 100, 2)), "%")

# Model f1-score: weighted average of Precision & Recall
print("f1-score:", (round(metrics.f1_score(yo_test, yo_pred, average='micro') * 100, 2)), '%')

# Area under ROC curve
print('roc_auc_score:', round(roc_auc_score(yo_test, yo_pred), 3))

Accuracy: 50.62 %
Precision: 50.62 %
Recall: 50.62 %
f1-score: 50.62 %
roc_auc_score: 0.506


# Logistic Regression with Stratified Sampling

In [80]:
# Splitting the dataset into train and test
from sklearn.model_selection import train_test_split
x3_train, x3_test, y3_train, y3_test = train_test_split(x, y, test_size = 0.30, stratify=y,  random_state =43)

In [81]:
# Build the logistic regression model with random sampling
from sklearn.linear_model  import LogisticRegression
AutoInsDT_S =LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
intercept_scaling=1, max_iter=100, multi_class='auto',
n_jobs=None, penalty='l2', random_state=None,
solver='lbfgs', tol=0.0001, verbose=0, warm_start=False)
AutoInsDT_S = AutoInsDT_S.fit(x3_train,y3_train)
AutoInsDT_S = AutoInsDT_S.fit(x3_train,y3_train)

In [82]:
# Predict the model with test data set
y3_pred = AutoInsDT_S.predict(x3_test)


In [83]:
# Display confusion matrix and classifiction report
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y3_test, y3_pred))
print(classification_report(y3_test, y3_pred))

[[34127     0]
 [ 9751     0]]
              precision    recall  f1-score   support

           0       0.78      1.00      0.87     34127
           1       0.00      0.00      0.00      9751

    accuracy                           0.78     43878
   macro avg       0.39      0.50      0.44     43878
weighted avg       0.60      0.78      0.68     43878



In [84]:
# Evaluate the model performance by metrics
from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score
# Model Accuracy: how often is the classifier correct?
print("Accuracy:", (round(metrics.accuracy_score(y3_test, y3_pred) * 100, 2)), "%")
# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:", (round(metrics.precision_score(y3_test, y3_pred) * 100, 2)), '%')
# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:", (round(metrics.recall_score(y3_test, y3_pred) * 100, 2)), "%")
# Model f1-score: weighted average of Precision & Recall
print("f1-score:", (round(metrics.f1_score(y3_test, y3_pred) * 100, 2)), '%')
# Area under ROC curve
print('roc_auc_score:', round(roc_auc_score(y3_test, y3_pred), 3))

Accuracy: 77.78 %
Precision: 0.0 %
Recall: 0.0 %
f1-score: 0.0 %
roc_auc_score: 0.5


# Logistic Regression with Under Sampling

In [85]:
# To build the 'Logistic Regression' model with random sampling

from sklearn.linear_model import LogisticRegression

AutoInsLR_U = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
intercept_scaling=1, max_iter=100, multi_class='auto',
n_jobs=None, penalty='l2', random_state=None,
solver='lbfgs', tol=0.0001, verbose=0, warm_start=False)

AutoInsLR_U = AutoInsLR_U.fit(xu_train,yu_train)

In [86]:

# Predict the model with test data set

yu_pred = AutoInsLR_U.predict(xu_test)

In [87]:
# Display confusion matrix and classifiction report

from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(yu_test, yu_pred))
print(classification_report(yu_test, yu_pred))

[[4979 4774]
 [4991 4758]]
              precision    recall  f1-score   support

           0       0.50      0.51      0.50      9753
           1       0.50      0.49      0.49      9749

    accuracy                           0.50     19502
   macro avg       0.50      0.50      0.50     19502
weighted avg       0.50      0.50      0.50     19502



In [88]:
# Evaluate the model performance by metrics

from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score

# Model Accuracy: how often is the classifier correct?
print("Accuracy:", (round(metrics.accuracy_score(yu_test, yu_pred) * 100, 2)), "%")

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:", (round(metrics.precision_score(yu_test, yu_pred, average='micro') * 100, 2)), '%')

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:", (round(metrics.recall_score(yu_test, yu_pred, average='micro') * 100, 2)), "%")

# Model f1-score: weighted average of Precision & Recall
print("f1-score:", (round(metrics.f1_score(yu_test, yu_pred, average='micro') * 100, 2)), '%')

# Area under ROC curve
print('roc_auc_score:', round(roc_auc_score(yu_test, yu_pred), 3))

Accuracy: 49.93 %
Precision: 49.93 %
Recall: 49.93 %
f1-score: 49.93 %
roc_auc_score: 0.499


# Logistic Regression - Combining Over Sampling and Under Sampling

In [89]:
# To build the 'Logistic Regression' model with random sampling

from sklearn.linear_model import LogisticRegression

AutoInsLR_C = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
intercept_scaling=1, max_iter=100, multi_class='auto',
n_jobs=None, penalty='l2', random_state=None,
solver='lbfgs', tol=0.0001, verbose=0, warm_start=False)
AutoInsLR_C = AutoInsLR_C.fit(xc_train,yc_train)

In [90]:
# Predict the model with test data set

yc_pred = AutoInsLR_C.predict(xc_test)


In [91]:
# Display confusion matrix and classifiction report

from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(yc_test, yc_pred))
print(classification_report(yc_test, yc_pred))

[[34004     0]
 [24012     0]]
              precision    recall  f1-score   support

           0       0.59      1.00      0.74     34004
           1       0.00      0.00      0.00     24012

    accuracy                           0.59     58016
   macro avg       0.29      0.50      0.37     58016
weighted avg       0.34      0.59      0.43     58016



In [92]:
# Evaluate the model performance by metrics

from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score

# Model Accuracy: how often is the classifier correct?
print("Accuracy:", (round(metrics.accuracy_score(yc_test, yc_pred) * 100, 2)), "%")

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:", (round(metrics.precision_score(yc_test, yc_pred, average='micro') * 100, 2)), '%')

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:", (round(metrics.recall_score(yc_test, yc_pred, average='micro') * 100, 2)), "%")

# Model f1-score: weighted average of Precision & Recall
print("f1-score:", (round(metrics.f1_score(yc_test, yc_pred, average='micro') * 100, 2)), '%')

# Area under ROC curve
print('roc_auc_score:', round(roc_auc_score(yc_test, yc_pred), 3))

Accuracy: 58.61 %
Precision: 58.61 %
Recall: 58.61 %
f1-score: 58.61 %
roc_auc_score: 0.5
